This notebook was used to filter comparative questions from SQuAD dataset. It can classsify any question as comparative or not.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def find_value_modifier(sentence):
    sentence = sentence.lower()
    value_modifier ={}
    text = nltk.tokenize.word_tokenize(sentence)
    tagged_list = nltk.pos_tag(text)
    for i, entry in enumerate(tagged_list):
        if entry[1] == 'CD':
            if i-1>=0 and tagged_list[i-1][0] in {"a","an","one"}:
                if i-2>=0 and tagged_list[i-2][1]=='JJR' :
                    modifier = tagged_list[i-1][0]
                    value_modifier[1] = modifier
                elif i-2>=0 and i-3>=0 and tagged_list[i-2][1]=='IN' and (tagged_list[i-3][1] =='JJR' or tagged_list[i-3][1] =='JJS'):
                    modifier = tagged_list[i-3][0] +" "+ tagged_list[i-2][0]
                    value_modifier[1] = modifier
                elif i-2>=0 and i-3>=0 and tagged_list[i-3][1]=='IN' and (tagged_list[i-2][1] =='JJR' or tagged_list[i-2][1] =='JJS'):
                    modifier = tagged_list[i-3][0] +" "+ tagged_list[i-2][0]
                    value_modifier[1] = modifier
                elif i-2>=0 and ( tagged_list[i-2][1]=='IN' or tagged_list[i-2][1]=='RB'): #over, before
                    modifier= tagged_list[i-2][0]
                    value_modifier[1] = modifier
            elif i-1>=0 and  tagged_list[i-1][0] in {"$"}:
                if i-2>=0 and  tagged_list[i-2][1]=='JJR' :
                    modifier = tagged_list[i-1][0]
                    try:
#                         if entry[0].isdigit():
                          value_modifier[float(entry[0].replace(',', ''))] = modifier
                    except:
                        continue
                elif i-2>=0 and i-3>=0 and tagged_list[i-2][1]=='IN' and (tagged_list[i-3][1] =='JJR' or tagged_list[i-3][1] =='JJS'):
                    modifier = tagged_list[i-3][0] +" "+ tagged_list[i-2][0]
                    try:
#                       if entry[0].isdigit():
                          value_modifier[float(entry[0].replace(',', ''))] = modifier
                    except:
                        continue

                elif i-2>=0 and (tagged_list[i-2][1]=='IN' or tagged_list[i-2][1]=='RB'): #over, before
                    modifier= tagged_list[i-2][0]
                    try:
#                         if entry[0].isdigit():
                            value_modifier[float(entry[0].replace(',', ''))] = modifier
                    except:
                        continue
            elif i-2>=0 and i-1>=0 and tagged_list[i-1][1]=='IN' and (tagged_list[i-2][1] =='JJR' or tagged_list[i-2][1] =='JJS'):
                modifier = tagged_list[i-2][0]+ " "+tagged_list[i-1][0]
                try:
                    value_modifier[float(entry[0].replace(',', ''))] = modifier
                except:
                    continue
                # at least, at most
            elif i-2>=0 and i-1>=0 and tagged_list[i-2][1]=='IN' and (tagged_list[i-1][1] =='JJR' or tagged_list[i-1][1] =='JJS'):
                modifier = tagged_list[i-2][0] +" "+tagged_list[i-1][0]
                try:
                    value_modifier[float(entry[0].replace(',', ''))] = modifier
                except:
                    continue
          #one word modifiers
            elif i-1>=0 and tagged_list[i-1][1]=='JJR' or tagged_list[i-1][1]=='JJS':
                modifier = tagged_list[i-1][0]
                try:
                    value_modifier[float(entry[0].replace(',', ''))] = modifier
                except:
                    continue
            # elif i-1>=0 and (tagged_list[i-1][1]=='IN' or tagged_list[i-1][1]=='RB'): #over, before,above
            #       modifier= tagged_list[i-1][0]
            #       try:
            #           value_modifier[float(entry[0].replace(',', ''))] = modifier
            #       except:
            #             continue
            if i+1<len(text) and i+2<len(text) and tagged_list[i+1][1]=='CC' and (tagged_list[i+2][1] =='JJR' or tagged_list[i+2][1] =='JJS'): #ore more
                modifier = tagged_list[i+1][0]+ " "+ tagged_list[i+2][0]
                try:
                    value_modifier[float(entry[0].replace(',', ''))] = modifier
                except:
                    continue
    return value_modifier

In [ ]:
import nltk
import re

def has_numbers(neighbors):
  for word in neighbors:
    if word.isnumeric():
      return True
  return False

def get_neighbors_4(id, sentence):     #considering only 4 neighbors in front of a word
  neighbors = []
  # for i in range(4):   #left neighbors
  #   if id-(4-i)>=0:
  #     neighbors.append(sentence[id-(4-i)])

  for i in range(4):   #right neighbors
    if id+i< len(sentence):
      neighbors.append(sentence[id+i])
  return neighbors

def check_4_CD(neighbors):               #checking for a CD POS tag in a list of tags with the words
  #print(neighbors)
  for token in neighbors:
    if(len(token)!=2):
      continue;
    if(token[1]=='CD'):
      return True
  return False

def get_tag_4(id, tokens):               #get 4 front neighbors for tags
  neighbors = []
  #print(f"The input tokens are {tokens}")
  for i in range(4):     #left neighbors
    if id-(4-i):
      neighbors.append(tokens[id-(4-i)])

  for i in range(8):
    if id+i< len(tokens):
      neighbors.append(tokens[id+i])
  return neighbors

def is_comparative(sentence):           #main function that checks if a question is comparative
    # Tokenize the sentence and perform POS tagging
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    #print(tagged_tokens)
    # Define a list of POS tags indicating comparative adjectives
    comparative_tags = ['JJR', 'RBR']
    comparative_tags_1 = ['JJ', 'JJR', 'JJS', 'RB', 'RBS', 'RBR', 'WRB']
    #comparative_tags_1 = ['JJR', 'RBR']
    #Check if any token has a comparative POS tag

    # specific_adjectives = ["taller", "heavier","more expensive"]

    # for adj in specific_adjectives:
    #     if adj in sentence.lower():
    #         return True

    for i in range(len(tagged_tokens)):     #taller than, smaller than etc
      if(i==len(tagged_tokens)-1):
        continue
      else:
        if(tagged_tokens[i][1] in comparative_tags and tagged_tokens[i+1][0]=="than"):
        #if(tagged_tokens[i][1] in comparative_tags):
          if(check_4_CD(get_tag_4(i, tagged_tokens))):    #adding to check if there is a CD in 4 neighbors right
            #print("type 1")
            return True

    for i in range(len(tagged_tokens)):    #more taller, less happier etc
      if(i==0):
        continue
      else:
        if(tagged_tokens[i][1] in comparative_tags_1 and (tagged_tokens[i-1][0]=="more" or tagged_tokens[i-1][0]=="less")):
          if(check_4_CD(get_tag_4(i, sentence))):
            #print("type 2")
            return True


    #hard coded patterns to quickly change the function

    # Check if there are specific comparative patterns
    comparative_patterns = [r'\bmore than\b',r'\bless than\b',r'\bmore than that of\b',r'\bless than that of \b',r'\bmore than a\b',r'\bless than a\b',r'\bmore than the\b',r'\bless than the\b',r'\bover\b',r'\bunder\b',r'\bgreater than or equal to\b', r'\bgreater than\b',r'\bless than or equal to\b', r'\bless than\b', r'\bexceeds\b', r'\bsurpasses\b', r'\bhigher than\b', r'\blower than\b', r'\babove\b', r'\bbeyond\b',r'\bupwards of\b',r'\bat least\b',r'\bno less than\b',r'\bat most\b',r'\bno more than\b',r'\bequal to or greater than\b',r'\bequal to or less than\b',r'\bwithin a range of\b',r'\bbetween\b',r'\bin excess of\b',r'\bnot below\b',r'\bnot above\b',r'\bover and above\b',r'\bup to\b',r'\bbeyond\b',r'\baround\b',r'\bapproximately\b',r'\babout\b',r'\balmost\b',r'\bclose to\b',r'\bnearly\b',r'\bexactly\b',r'\bapproximately\b',r'\broughly\b',r'\baround\b',r'\bnot less than\b']

    #returning true for 'less than a' and 'more than a'
    if(re.search(comparative_patterns[4], sentence, flags=re.IGNORECASE) or re.search(comparative_patterns[5], sentence, flags=re.IGNORECASE) or re.search(comparative_patterns[6], sentence, flags=re.IGNORECASE) or re.search(comparative_patterns[7], sentence, flags=re.IGNORECASE)):
      #print("type 3")
      return True

    # Check if any comparative pattern is present in the sentence
    for pattern in comparative_patterns:
        matched = re.search(pattern, sentence, flags=re.IGNORECASE)
        if matched:
          neighbors = get_neighbors_4(int(matched.start()),sentence)
          #print(neighbors)
          if(has_numbers(neighbors)):
            #print("type 4")
            return True

    return False

# Example usage:
sentence1 = "The temperature is more than 30 degrees."
sentence2 = "This book is interesting."
sentence3 = "Less than 20% of the population voted."

print(is_comparative(sentence1))
print(is_comparative(sentence2))
print(is_comparative(sentence3))
print(is_comparative("I would like to check if I am taller than 5ft"))
print(is_comparative("I would rather do it today than yesterday"))
print(is_comparative("I like you better than her"))


True
False
True
False
False
False


In [ ]:
print(is_comparative("I would like you thank you more than enough number of times."))

False


In [ ]:
print(is_comparative("Which mountains are taller than 500ft?"))
print(is_comparative("Rachit is a very studious boy"))
print(is_comparative("You think you are better than me?"))
print(is_comparative("Utsav's height is 5.9ft, Vishnu's height is 5.11ft. Utsav is taller than Vishnu"))
print(is_comparative("Which city had more crime between 2014 and 2015 than michigan?"))
print(is_comparative("I am 2 ounces lighter today"))

True
False
False
True
False
False


In [ ]:
print(is_comparative("what % of households were spending less than £20 per week in 2004"))

True


In [ ]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
import json

--2023-11-30 13:02:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2023-11-30 13:02:56 (296 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2023-11-30 13:02:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [ ]:
with open('train-v2.0.json', 'rb') as f:
  raw_contexts = []
  raw_questions = []
  raw_answers = []
  raw_ids = []
  raw_answer_starts = []
  raw_data = json.load(f)
  questions = []
  for group in raw_data['data']:
    for paragraph in group['paragraphs']:
      context = paragraph['context']
      for qa in paragraph['qas']:
        question = qa['question']
        for answer in qa['answers']:
          raw_contexts.append(context.lower())
          raw_questions.append(question.lower())
          raw_answers.append(answer)
          raw_ids.append(qa['id'])


In [ ]:
with open('dev-v2.0.json', 'rb') as g:
  raw_data = json.load(g)
  questions = []
  for group in raw_data['data']:
    for paragraph in group['paragraphs']:
      context = paragraph['context']
      for qa in paragraph['qas']:
        question = qa['question']
        for answer in qa['answers']:
          raw_contexts.append(context.lower())
          raw_questions.append(question.lower())
          raw_answers.append(answer)
          raw_ids.append(qa['id'])

In [ ]:
print(len(raw_contexts))
print(len(raw_questions))
print(len(raw_answers))
print(len(raw_ids))

107123
107123
107123
107123


In [ ]:
final_questions = []
final_answers = []
final_ids = []
final_contexts = []

In [ ]:
new_list = ['for', 'in', 'at', 'of']

In [ ]:
for i in range(len(raw_answers)):
  tempor = find_value_modifier(raw_questions[i])
  if(is_comparative(raw_questions[i]) or (tempor!={} and (list(tempor.values())[0] not in new_list))):
    if(len(raw_questions)!=0) and raw_questions[-1]!=raw_questions[i]:
      final_questions.append(raw_questions[i])
      final_answers.append(raw_answers[i])
      final_ids.append(raw_ids[i])
      final_contexts.append(raw_contexts[i])
      print(raw_questions[i])

when did jay z and beyoncé become the first music couple worth over a billion dollars?
only a few of chopin's pieces involve more than the piano, including piano concertos, songs and what?
in the 1930s, new york city had more than 10 million people becoming the first what in history?
how many buildings located in nyc are at least 330 feet in height?
in what year did the city have less than 500 homicides?
what drives the plot of the book more than the characters?
less than 1% of the world's total grid electricity was generated by solar energy in what year?
a program in los angeles believes that with $1 billion, city temperatures could be reduced by approximately how many degrees in celsius?
which celebrity sued west in 2006 for trademark infrigement?
what buddhist movement has linked more than 12 million people?
how many contestants had at least one million radio spins by 2010?
what was the name for a pub that could sell beer from more than one brewery?
what % of households were spendin

In [ ]:
# for i in range(len(raw_answers)):
#   tempor = find_value_modifier(raw_questions[i])
#   #print(f"tempor values {tempor.values()}")
#   if(tempor!={}):
#     #print(type(tempor))
#     if(list(tempor.values())[0] not in new_list):
#       print(tempor.values())
#       final_questions.append(raw_questions[i])
#       final_answers.append(raw_answers[i])
#       final_ids.append(raw_ids[i])
#       final_contexts.append(raw_contexts[i])

In [ ]:
print(len(final_questions))

243


In [ ]:
print(final_answers[0])

{'text': '2013', 'answer_start': 1204}


In [ ]:
def stripped_question(question):
  return question.replace("\"", "")

In [ ]:
for i in range(len(final_questions)):

  print("{")
  print(f"\"id\" : \"{final_ids[i]}\",")
  print(f"\"sentence\" : \"{stripped_question(final_contexts[i])}\",")
  print(f"\"question\" : \"{stripped_question(final_questions[i])}\",")
  print(f"\"answer\" : \"{stripped_question(final_answers[i]['text'])}\",")
  print(f"\"answer_start\" : {final_answers[i]['answer_start']},")
  print(f"\"answer_end\" : 0,")

  if(final_answers[i]['answer_start']=="-1"):
    print("\"is_impossible\": true")
  else:
    print("\"is_impossible\": false")

  print("},")


{
"id" : "56d4d8702ccc5a1400d83298",
"sentence" : "forbes magazine began reporting on beyoncé's earnings in 2008, calculating that the $80 million earned between june 2007 to june 2008, for her music, tour, films and clothing line made her the world's best-paid music personality at the time, above madonna and celine dion. they placed her fourth on the celebrity 100 list in 2009 and ninth on the most powerful women in the world list in 2010. the following year, forbes placed her eighth on the best-paid celebrities under 30 list, having earned $35 million in the past year for her clothing line and endorsement deals. in 2012, forbes placed beyoncé at number 16 on the celebrity 100 list, twelve places lower than three years ago yet still having earned $40 million in the past year for her album 4, clothing line and endorsement deals. in the same year, beyoncé and jay z placed at number one on the world's highest-paid celebrity couples, for collectively earning $78 million. the couple made i

In [ ]:
# for i in range(0, 5000):
#   print(raw_questions[i])

In [ ]:
# for i in range(5000, 10000):
#   print(raw_questions[i])

In [ ]:
qty_que = []

In [ ]:
# for que in raw_questions:
#   tokens = nltk.word_tokenize(que)
#   tagged_tokens = nltk.pos_tag(tokens)
#   if(check_4_CD(tagged_tokens)):
#     qty_que.append(que)


In [ ]:
print(len(qty_que))

0


In [ ]:
# for i in range(0, 5000):
#   print(qty_que[i])

In [ ]:
# for i in range(5000, 10000):
#   print(qty_que[i])

In [ ]:
# for i in range(10000, 12228):
#   print(qty_que[i])

In [ ]:
print(find_value_modifier("Which mountains are taller than 500ft?"))
print(find_value_modifier("The temperature is more than 30 degrees."))
print(find_value_modifier("Less than 20% of the population voted."))


{}
{30.0: 'more than'}
{20.0: 'less than'}


In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')